In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 76.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 39.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:000:00:01

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-0.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0.2, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 24 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
sql_prompt = """You are a SQL expert. Convert the following natural language question to SQL based on the schema below.
### SCHEMA:
CREATE TABLE categories (
    category_id CHAR(36) PRIMARY KEY DEFAULT (UUID()), 
    name VARCHAR(100) NOT NULL UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE manufacturers (
    manufacturer_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    name VARCHAR(100) NOT NULL UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE products (
    product_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    name VARCHAR(100) NOT NULL UNIQUE, 
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE stock (
    stock_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    product_id CHAR(36) NOT NULL,
    stock_quantity INT NOT NULL CHECK (stock_quantity >= 0),
    manufacturer_id CHAR(36) NOT NULL,
    category_id CHAR(36),
    FOREIGN KEY (product_id) REFERENCES products (product_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (manufacturer_id) REFERENCES manufacturers (manufacturer_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (category_id) REFERENCES categories (category_id) ON DELETE SET NULL ON UPDATE CASCADE
);

CREATE TABLE product_prices (
    price_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    stock_id CHAR(36) NOT NULL,
    price_type ENUM('import', 'export') NOT NULL,
    price DECIMAL(10, 2) NOT NULL CHECK (price > 0),
    effective_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (stock_id) REFERENCES stock (stock_id) ON DELETE CASCADE ON UPDATE CASCADE
);

CREATE TABLE partners (
    partner_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    name VARCHAR(100) NOT NULL UNIQUE,
    partner_type ENUM('supplier', 'customer') NOT NULL,
    address TEXT,
    phone VARCHAR(15) NOT NULL,
    email VARCHAR(100),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE employees (
    employee_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL UNIQUE,
    phone VARCHAR(15),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE transactions (
    transaction_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    action ENUM('import', 'export') NOT NULL,
    stock_id CHAR(36) NOT NULL,
    partner_id CHAR(36) NOT NULL,
    employee_id CHAR(36) NOT NULL,
    price_per_unit DECIMAL(10, 2) NOT NULL,
    quantity INT NOT NULL CHECK (quantity > 0),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (stock_id) REFERENCES stock (stock_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (partner_id) REFERENCES partners (partner_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (employee_id) REFERENCES employees (employee_id) ON DELETE CASCADE ON UPDATE CASCADE
);

### QUESTION:
{}

### RESPONSE:
{}
"""

In [5]:
EOS_TOKEN = tokenizer.eos_token 

def formatting_prompts_func(examples):
    questions = examples['question']
    outputs = examples['sql']
    texts = []
    for question, output in zip(questions, outputs):
        # Định dạng prompt và thêm EOS token
        text = sql_prompt.format(question, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

from datasets import load_dataset
dataset = load_dataset('gianghp/ttcs-sql-dataset', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

ttcs-dataset.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3029 [00:00<?, ? examples/s]

Map:   0%|          | 0/3029 [00:00<?, ? examples/s]

In [6]:
dataset[0]

{'question': "Get the total number of transactions for 'Anker Soundcore 3'.",
 'sql': "SELECT COUNT(*) FROM transactions t JOIN stock s ON t.stock_id = s.stock_id JOIN products p ON s.product_id = p.product_id WHERE p.name = 'Anker Soundcore 3'",
 'text': "You are a SQL expert. Convert the following natural language question to SQL based on the schema below.\n### SCHEMA:\nCREATE TABLE categories (\n    category_id CHAR(36) PRIMARY KEY DEFAULT (UUID()), \n    name VARCHAR(100) NOT NULL UNIQUE,\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\nCREATE TABLE manufacturers (\n    manufacturer_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),\n    name VARCHAR(100) NOT NULL UNIQUE,\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\nCREATE TABLE products (\n    product_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),\n    name VARCHAR(100) NOT NULL UNIQUE, \n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\nCREATE TABLE stock (\n    stock_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),\n 

In [7]:
dataset[0]["text"]

"You are a SQL expert. Convert the following natural language question to SQL based on the schema below.\n### SCHEMA:\nCREATE TABLE categories (\n    category_id CHAR(36) PRIMARY KEY DEFAULT (UUID()), \n    name VARCHAR(100) NOT NULL UNIQUE,\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\nCREATE TABLE manufacturers (\n    manufacturer_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),\n    name VARCHAR(100) NOT NULL UNIQUE,\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\nCREATE TABLE products (\n    product_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),\n    name VARCHAR(100) NOT NULL UNIQUE, \n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\nCREATE TABLE stock (\n    stock_id CHAR(36) PRIMARY KEY DEFAULT (UUID()),\n    product_id CHAR(36) NOT NULL,\n    stock_quantity INT NOT NULL CHECK (stock_quantity >= 0),\n    manufacturer_id CHAR(36) NOT NULL,\n    category_id CHAR(36),\n    FOREIGN KEY (product_id) REFERENCES products (product_id) ON DELETE CASCADE ON UPDATE CAS

In [8]:
dataset = dataset.train_test_split(test_size=0.2)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'sql', 'text'],
        num_rows: 2423
    })
    test: Dataset({
        features: ['question', 'sql', 'text'],
        num_rows: 606
    })
})

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2
)

num_gpus = torch.cuda.device_count()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=False, 
    args=TrainingArguments(
        output_dir='outputs',
        per_device_train_batch_size=8,  
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        optim="adamw_8bit",
        num_train_epochs = 7,
        eval_strategy="steps",
        eval_steps=0.05,
        save_strategy="steps",
        save_steps=0.05,
        logging_steps=5,  
        weight_decay=0.01,
        seed=3407,
        lr_scheduler_type="cosine",  
        warmup_ratio=0.05,
        learning_rate=2e-4,
        fp16 = True,
        bf16 = False,
        dataloader_num_workers=4,  
        report_to="none",
        load_best_model_at_end=True,
        save_safetensors=False
    ),
    callbacks=[early_stopping_callback],
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/2423 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/606 [00:00<?, ? examples/s]

In [11]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.223 GB of memory reserved.


In [12]:
from unsloth import unsloth_train

train_stat = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,423 | Num Epochs = 7 | Total steps = 266
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 35,192,832/529,225,600 (6.65% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
14,0.207900,0.057207
28,0.040300,0.037741
42,0.033000,0.031587
56,0.027600,0.028805
70,0.026400,0.026705
84,0.022900,0.025838
98,0.022600,0.024784
112,0.022000,0.024109
126,0.018800,0.023740
140,0.019600,0.023573


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [13]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    sql_prompt.format(
        "List name and stock quantity of all products come from 'legion'",
        ''
    )

], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256, eos_token_id=tokenizer.eos_token_id, do_sample=False)

SELECT p.name, SUM(s.stock_quantity) FROM products p JOIN stock s ON p.product_id = s.product_id JOIN manufacturers m ON s.manufacturer_id = m.manufacturer_id WHERE m.name = 'legion' GROUP BY p.name, s.stock_quantity
<|im_end|>
